## pysparkTJhw08
用DataFrame API 完成发现共同好友和好友推荐

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("pysparkTJhw07") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [3]:
df1 = spark.read.csv("file:///home/ian/code/github/LSCJcourses/pysparkTJ/friends.txt", sep='\t', header=False)
df1.show(5)

+-----+------+
|  _c0|   _c1|
+-----+------+
|11342|387543|
|11342|427436|
|11342|538214|
|11342|638706|
|11342|645018|
+-----+------+
only showing top 5 rows



In [4]:
df1.registerTempTable('f1')

In [5]:
spark.sql('select count(*) from f1').show()

+--------+
|count(1)|
+--------+
|    3769|
+--------+



In [6]:
#spark 分组后字符串拼接
df2 = spark.sql('select _c0, concat_ws(" ",collect_set(_c1)) t1  from f1 GROUP BY _c0')
df2.show(5)

+------+--------------------+
|   _c0|                  t1|
+------+--------------------+
|465399|251691 855832 659...|
|174836|507854 226374 877041|
|     7|19576 514170 4334...|
|805694|185067 21 711442 ...|
|909316|806291 876328 756...|
+------+--------------------+
only showing top 5 rows



In [7]:
df3 = spark.sql('select _c0 id, collect_set(_c1) friends  from f1 GROUP BY id')
df3.show(5)

+------+--------------------+
|    id|             friends|
+------+--------------------+
|465399|[251691, 855832, ...|
|174836|[507854, 226374, ...|
|     7|[19576, 514170, 4...|
|805694|[185067, 21, 7114...|
|909316|[806291, 876328, ...|
+------+--------------------+
only showing top 5 rows



In [8]:
import itertools

In [9]:
rdd1 = df3.rdd.flatMap(lambda x:[(i,x[0]) for i in list(itertools.combinations(x[1],2))])

In [10]:
df4 = rdd1.toDF()
df4.show(5)

+----------------+------+
|              _1|    _2|
+----------------+------+
|[251691, 855832]|465399|
|[251691, 659637]|465399|
|[251691, 271187]|465399|
|[251691, 476261]|465399|
|[251691, 126013]|465399|
+----------------+------+
only showing top 5 rows



In [11]:
df5 = df4.groupBy('_1').agg({'_2':'collect_set'})
df5.show()

+----------------+--------------------+
|              _1|     collect_set(_2)|
+----------------+--------------------+
|[121202, 700840]|    [738994, 465399]|
|[126108, 366358]|    [738994, 465399]|
|[251691, 575717]|            [465399]|
|[741151, 803349]|            [465399]|
|[104279, 317571]|[806291, 876328, ...|
|[460382, 771890]|                [15]|
|[537900, 616640]|[621034, 151230, 28]|
|[635575, 648208]|[748615, 862566, ...|
|[291224, 430708]|            [818382]|
|[291879, 202128]|            [502809]|
|[310385, 417121]|            [502809]|
|[215367, 479713]|[51633, 891240, 3...|
|[721816, 490263]|[215367, 479713, ...|
|[116209, 204549]|[40033, 202339, 8...|
| [396121, 12199]|            [498271]|
|[455271, 285042]|[297289, 241611, ...|
|[494042, 528391]|     [430119, 30957]|
|[396121, 390239]|            [781983]|
|[396121, 431311]|            [781983]|
|[166776, 104370]|                [31]|
+----------------+--------------------+
only showing top 20 rows

